# Day 18

In [5]:
# Definitions

import math
from functools import reduce
from itertools import permutations

def parse_input(lines):
    return [eval(line) for line in lines.strip().split('\n')]

def split(n):
    x = n / 2
    return [math.floor(x), math.ceil(x)]

class Element:
    def __init__(self, *, parent, is_right, is_pair, depth):
        self.parent = parent
        self.is_right = is_right
        self.is_pair = is_pair
        self.depth = depth        

class Value(Element):
    def __init__(self, value, *, parent, is_right, depth):
        super().__init__(parent=parent, is_right=is_right, is_pair=False, depth=depth)
        self.value = value
        
    def __repr__(self):
        return repr(self.value)
    
    def to_el(self):
        return self.value
    
    def add_from_left(self, n):
        self.value += n
    
    def add_from_right(self, n):
        self.value += n
        
    def split(self):
        if self.value >= 10:
            self.parent.replace_child(self, split(self.value))
            return True
        
    def explode(self):
        return False
    
    def _flattener(self):
        yield self
        
    def magnitude(self):
        return self.value

class Pair(Element):
    def __init__(self, left_child, right_child, *, parent=None, is_right=False, depth=0):
        super().__init__(parent=parent, is_right=is_right, is_pair=True, depth=depth)
        self.left_child = self._make_child(left_child, False)
        self.right_child = self._make_child(right_child, True)
    
    def __repr__(self):
        return f'[{repr(self.left_child)},{repr(self.right_child)}]'
    
    def to_el(self):
        return [self.left_child.to_el(), self.right_child.to_el()]
    
    def add_from_left(self, n):
        self.left_child.add_from_left(n)
    
    def add_from_right(self, n):
        self.right_child.add_from_right(n)
        
    def split(self):        
        return any(child.split() for child in (self.left_child, self.right_child))
        
    def explode(self):
        if self.depth >= 4:
            return self.parent.replace_child(self, 0)
        else:
            return any(child.explode() for child in (self.left_child, self.right_child))
    
    def flatten(self):
        return [*self._flattener()]
    
    def _flattener(self):
        if self.is_simple():
            yield self
        else:
            yield from self.left_child._flattener()
            yield from self.right_child._flattener()
        
    def is_simple(self):
        return not (self.left_child.is_pair or self.right_child.is_pair)
    
    def replace_child(self, child, val):
        new_child = self._make_child(val, child.is_right)
        if new_child.is_right:
            self.right_child = new_child
        else:
            self.left_child = new_child
        return new_child
    
    def _make_child(self, child, is_right):
        if type(child) is list:
            return Pair(*child, parent=self, is_right=is_right, depth=self.depth+1)
        else:
            return Value(child, parent=self, is_right=is_right, depth=self.depth+1)
    
    def magnitude(self):
        return 3 * self.left_child.magnitude() + 2 * self.right_child.magnitude()
        
def try_explode(arr):
    for i, el in enumerate(arr):
        new_el = el.explode()        
        if new_el:            
            arr[i] = new_el            
            if i > 0:
                arr[i-1].add_from_right(el.left_child.value)                
            i2 = i + 1
            if i2 < len(arr):
                arr[i2].add_from_left(el.right_child.value)
            return True
    return False

def try_split(arr):
    for i, el in enumerate(arr):
        new_el = el.split()        
        if new_el:            
            arr[i] = new_el            
            return True
    return False

def reduce_pair(pair):    
    while True:
        arr = pair.flatten()
        if not try_explode(arr) and not try_split(arr):
            break
        
def add_arrs(left, right, *, to_el=True):
    if type(left) is not list:
        return right
    else:
        pair = Pair(left, right)
        reduce_pair(pair)
        if to_el:
            return pair.to_el()
        else:
            return pair
    
def find_largest_magnitude(arr):
    return max(add_arrs(*pair, to_el=False).magnitude() for pair in permutations(arr, 2))

In [7]:
# Use for demo input

lines = '''
[[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]
[[[5,[2,8]],4],[5,[[9,9],0]]]
[6,[[[6,2],[5,6]],[[7,6],[4,7]]]]
[[[6,[0,7]],[0,9]],[4,[9,[9,0]]]]
[[[7,[6,4]],[3,[1,3]]],[[[5,5],1],9]]
[[6,[[7,3],[3,2]]],[[[3,8],[5,7]],4]]
[[[[5,4],[7,7]],8],[[8,3],8]]
[[9,3],[[9,9],[6,[4,9]]]]
[[2,[[7,7],7]],[[5,8],[[9,3],[0,2]]]]
[[[[5,2],5],[8,[3,7]]],[[5,[7,5]],[4,4]]]
'''
arr = parse_input(lines)

# Part 1
result = reduce(add_arrs, arr)
print(result)
print(Pair(*result).magnitude())

# Part 2
print('-' * 76)
print(find_largest_magnitude(arr))


[[[[6, 6], [7, 6]], [[7, 7], [7, 0]]], [[[7, 7], [7, 7]], [[7, 8], [9, 9]]]]
4140
----------------------------------------------------------------------------
3993


In [8]:
# Use for actual input

with open('day18.txt', 'r') as f:
    arr = parse_input(f.read())
    
# Part 1
result = reduce(add_arrs, arr)
print(result)
print(Pair(*result).magnitude())

# Part 2
print('-' * 76)
print(find_largest_magnitude(arr))

[[[[7, 7], [7, 7]], [[8, 7], [8, 8]]], [[[8, 0], [8, 8]], [[8, 9], [9, 4]]]]
4435
----------------------------------------------------------------------------
4802
